# Fichier pour le nettoyage du jeu de données

## Importation des modules et du dataset

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("dataset.csv")

Jeu de données constitué des variables suivantes : 

- l’indice de masse corporel (bmi) : ça permet de donner un rapport entre la taille et le poids. Idéalement, il faut être en 28.5 et 24.9
- le sexe (sex): le genre de la personne qui contracte l'assurance, homme ou femme
- l’âge (age): l'âge du principal bénéficiaire
- le nombre d’enfant à charge (children) : Nombre d'enfant couverts par l'assurance
- smoker : fumeur ou non-fumeur
- région (region) : le zone résidentielle dans les US, nord-est, sud-est, sud-ouest, nord-ouest
- charges : la prime d’assurance facturée (cible)
​

## Première description globale du jeu de données

In [2]:
display(data.shape)
display(data.dtypes)

(1338, 7)

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

Nous avons un jeu de données composé de 7 variables et de 1338 observations.
Parmi ces variables, nous avons :
- deux variables quantitatives discrètes (age, children)
- deux variables quantitatives continues (bmi, charges)
- trois variables qualitatives nominales (sex, smoker, region)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


En ce qui concerne les observations, on remarque qu'il n'y a aucune valeur nulle dans le jeu de données.

In [4]:
duplicates = []
duplicates.append(data.loc[data[["charges"]].duplicated(keep=False),:])
duplicates.append(data.loc[data[["age", "sex", "bmi", "children", "smoker", "region"]].duplicated(keep=False),:])

for duplicate in duplicates:
    display(duplicate)

,age,sex,bmi,children,smoker,region,charges
195,19,male,30.59,0,no,northwest,1639.5631
581,19,male,30.59,0,no,northwest,1639.5631


,age,sex,bmi,children,smoker,region,charges
102,18,female,30.115,0,no,northeast,21344.84670
181,18,female,38.280,0,no,southeast,1631.82120
195,19,male,30.590,0,no,northwest,1639.56310
354,18,female,38.280,0,no,southeast,14133.03775
471,18,female,30.115,0,no,northeast,2203.47185
581,19,male,30.590,0,no,northwest,1639.56310


On teste les doublons de deux manières : 
- La première pour voir si certaines observations ont les mêmes charges
- La seconde pour voir si certaines observations ont le même age, sex, bmi, children, smoker et region

On remarque un seul vrai doublon (même valeur pour chaque variable) présent dans les deux tests.

Dans le second test on remarque aussi deux paires d'observations, ou chaque paire a les mêmes valeurs à l'exception des charges ou pour l'une des observations les charges sont 10x plus élevées.

In [7]:
data.drop_duplicates(subset=["charges"], inplace=True, ignore_index=True)
data.shape

(1337, 7)

On supprime l'une des observations doublons, ce qui porte notre nombre total d'observations à 1337.

## Description et nettoyage des différentes variables du jeu de données

### Variable age

In [6]:
data["age"].describe()

count    1337.000000
mean       39.222139
std        14.044333
min        18.000000
25%        27.000000
50%        39.000000
75%        51.000000
max        64.000000
Name: age, dtype: float64

A première vue, il n'y a pas de valeur aberrante pour la variable age.

### Variable sex

In [23]:
display(data["sex"].value_counts())

sex
male      675
female    662
Name: count, dtype: int64

Pas d'erreur apparente pour cette variable.

### Variable bmi

Rappel concernant l'IMC :

- Inférieur à 18.5 = insuffisance pondérale
- Entre 18.5 et 25 = poids normal
- Entre 25 et 30 = surpoids
- Entre 30 et 35 = obésité classe I (modérée)
- Entre 35 et 40 = obésité classe II (sévère)
- Supérieur à 40 = obésité classe III (massive)

In [10]:
display(data["bmi"].describe())

count    1337.000000
mean       30.663452
std         6.100468
min        15.960000
25%        26.290000
50%        30.400000
75%        34.700000
max        53.130000
Name: bmi, dtype: float64

On peut voir ici une valeur atypique pour la valeur maximale (53), mais ça ne parait pas aberrant lorsque l'on remarque que la moyenne est situé en obésité de classe I.

### Variable children

In [20]:
display(data["children"].value_counts())

children
0    573
1    324
2    240
3    157
4     25
5     18
Name: count, dtype: int64

On ne remarque pas de valeurs aberrantes pour cette variable.

### Variable smoker

In [22]:
display(data["smoker"].value_counts())

smoker
no     1063
yes     274
Name: count, dtype: int64

Pas d'erreur apparente pour la variable smoker.

### Variable region

In [25]:
display(data["region"].value_counts())

region
southeast    364
southwest    325
northwest    324
northeast    324
Name: count, dtype: int64

Pas d'erreur apparente pour la variable region.

### Variable charges

In [26]:
display(data["charges"].describe())

count     1337.000000
mean     13279.121487
std      12110.359656
min       1121.873900
25%       4746.344000
50%       9386.161300
75%      16657.717450
max      63770.428010
Name: charges, dtype: float64

Pour la variable charges on remarque une valeur atypique concernant la valeur maximale (63 770), surtout comparé au 3ème quartile qui est égale à 16 657, ce qui représente une différence de près de 50 000.

In [30]:
third_quartile = data.loc[data["charges"]>16675,:]
display(third_quartile["charges"].describe())

count      334.000000
mean     31151.710338
std      10581.240592
min      16776.304050
25%      21345.811525
50%      28936.803060
75%      39727.090013
max      63770.428010
Name: charges, dtype: float64

En isolant les observations dont les charges sont supérieur au 3ème quartile, on remarque que les valeurs sont assez étalées. 
Le 3ème quartile pour ce sous-échantillon est égale à 39 727. La valeur maximale s'apparente donc plus à une valeur atypique qu'à une valeur aberrante.

## Conclusion du nettoyage et chargement du dataset dans un nouveau csv

Le nettoyage du jeu de données n'a pour le moment nécessité que la suppression d'une ligne de doublon.

In [ ]:
data.to_csv("dataset_cleaned.csv")